<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/NLP/4_Seq2Seq/Seq2Seq.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>

# Modelos de Secuencia a Secuencia

Los modelos de secuencia a secuencia son modelos de aprendizaje profundo que han logrado mucho éxito en tareas como traducción automática, resumen de texto y subtítulos de imágenes. Google Translate comenzó a usar un modelo de este tipo en producción a finales de 2016. 

Un modelo de secuencia a secuencia es un modelo que toma una secuencia de elementos (palabras, letras, características de una imagen, etc.) y genera otra secuencia de elementos. 

![Imgur](https://i.imgur.com/nNOUbuN.gif)

# Arquitectura Encoder-Decoder

Para manejar este tipo de entradas y salidas, podemos diseñar una arquitectura con dos componentes principales.

1. El primer componente es un `Encoder` (codificador): toma una secuencia de longitud variable como entrada y la transforma en un estado (también llamado contexto) con una forma fija. 
2. El segundo componente es un `Decoder` (encoder): mapea el estado codificado de una forma fija a una secuencia de longitud variable.


![Imgur](https://i.imgur.com/dd2Qril.gif)

En las siguientes celdas vamos a definir una interfaz para el decoder y otra para el decoder. Las interfaces son clases que establecen las responsabilidades básicas de un objeto, pero que dejan que cada objeto decida como implementarlas.

Como vemos en la siguiente celda un Encoder es un modelo que recibe una entrada secuencial X (y algún otro argumento opcional) y genera una salida.

In [49]:
import torch
import torch.nn as nn
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()

    # Más tarde puede haber argumentos adicionales
    # (por ejemplo, longitud para excluir el relleno)
    def forward(self, X, *args):
        raise NotImplementedError

Por otro lado, el decoder debe tener una función adicional `init_state` para convertir la salida del encoder en el vector estado codificado. Y el forward debe recibir el estado codificado y la entrada X.

In [50]:
class Decoder(nn.Module):
    """The base decoder interface for the encoder-decoder architecture."""
    def __init__(self):
        super().__init__()

    # Más tarde puede haber argumentos adicionales
    # (por ejemplo, longitud para excluir el relleno)
    def init_state(self, enc_outputs, *args):
        raise NotImplementedError

    def forward(self, X, state):
        raise NotImplementedError

Al final, la arquitectura Encoder-Decoder contiene tanto un encoder como un decoder, con argumentos adicionales opcionales. En la función forward, la salida del encoder se usa para producir el estado codificado, y el decoder usará este estado como una de sus entradas.

In [51]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_X, dec_X, *args):
        enc_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_outputs, *args)
        # Sólo devuelve la salida del decoder
        return self.decoder(dec_X, dec_state)[0]

#Traducción Automática (Neuronal)

La traducción automática se refiere a la transformación automática de una secuencia de un idioma a otro. Durante décadas, los enfoques estadísticos habían dominado este campo antes del surgimiento del aprendizaje de extremo a extremo utilizando redes neuronales. Esta última a menudo se denomina traducción automática neuronal para distinguirse de la traducción automática estadística que implica el análisis estadístico en componentes como un modelo de traducción y un modelo de lenguaje. 



La traducción automática neuronal se implementa como un modelo de secuencia a secuencia donde el encoder y el decoder son RNNs y el contexto se obtiene a partir de las variables ocultas.

En la siguiente visualización, cada pulso para el encoder o decoder es esta RNN procesando sus entradas y generando una salida para ese paso de tiempo. Dado que tanto el encoder como el decoder son RNN, cada vez que el paso uno de los RNN realiza algún procesamiento, actualiza su estado oculto en función de sus entradas y las entradas anteriores que ha visto.

![Imgur](https://i.imgur.com/6AUERAx.gif)

Veamos los estados ocultos del encoder. Observe cómo el último estado oculto es en realidad el contexto que pasamos al decoder. El decoder también mantiene un estado oculto que pasa de un paso de tiempo al siguiente. Simplemente no lo visualizamos en este gráfico porque estamos interesados en las partes principales del modelo por ahora.

## Encoder


Técnicamente hablando, el encoder transforma una secuencia de entrada de longitud variable en una variable de contexto de forma fija $\mathbf{c}$ y codifica la información de la secuencia de entrada en esta variable de contexto. Como se muestra en las figuras anteriores, podemos usar un RNN para diseñar dicho encoder.

Consideremos un ejemplo de secuencia (tamaño de lote: 1). Suponga que la secuencia de entrada es $x_1, \ldots, x_T$
, tal que $x_t$ es el $t^{\mathrm{ésimo}}$ token en la secuencia de texto de entrada. En el paso de tiempo $t$, la RNN transforma el vector de características de entrada $\mathbf{x}_t$ para $x_t$ y el estado oculto $\mathbf{h} _{t-1}$ del paso de tiempo anterior en el estado oculto actual. Podemos usar una función para expresar la transformación de la capa recurrente de la RNN:

$$\mathbf{h}_t = f(\mathbf{x}_t, \mathbf{h}_{t-1}). $$

En general, el encoder transforma los estados ocultos en todos los pasos de tiempo en la variable de contexto a través de una función personalizada $q$

$$\mathbf{c} =  q(\mathbf{h}_1, \ldots, \mathbf{h}_T).$$

Por ejemplo, al elegir $q(\mathbf{h}_1, \ldots, \mathbf{h}_T) = \mathbf{h}_T$ como en la figura de la sección anterior, la variable de contexto es solo el estado oculto $\mathbf{h}_T$
de la secuencia de entrada en el paso de tiempo final.

Hasta ahora, hemos utilizado un RNN unidireccional para diseñar el encoder, donde un estado oculto solo depende de la subsecuencia de entrada hasta el paso de tiempo actual del estado oculto. También podemos construir encoders usando RNN bidireccionales. En este caso, un estado oculto depende de la subsecuencia anterior y posterior al paso de tiempo (incluida la entrada en el paso de tiempo actual), que codifica la información de toda la secuencia.

Ahora implementemos el encoder RNN. Tenga en cuenta que usamos una capa de embedding para obtener el vector de características para cada token en la secuencia de entrada. El peso de una capa de embedding es una matriz cuyo número de filas es igual al tamaño del vocabulario de entrada (vocab_size) y el número de columnas es igual a la dimensión del vector de características (embed_size). Para cualquier índice de token de entrada $i$, la capa de embedding obtiene la fila (a partir de 0) de la matriz de peso para devolver su vector de características. Además, aquí elegimos una GRU multicapa para implementar el encoder.

In [52]:
def init_RNN(module):
    if type(module) == nn.Linear:
         nn.init.xavier_uniform_(module.weight)
    if type(module) == nn.GRU:
        for param in module._flat_weights_names:
            if "weight" in param:
                nn.init.xavier_uniform_(module._parameters[param])

class RNNEncoder(Encoder):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers, dropout=dropout)
        self.hid_dim = num_hiddens
        self.n_layers = num_layers
        self.apply(init_RNN)

    def forward(self, X, *args):
        # X shape: (num_steps, batch_size)
        embs = self.embedding(X.type(torch.int64))
        # embs shape: (num_steps, batch_size, embed_size)
        output, state = self.rnn(embs)
        # recuerde que output devolverá siempre el valor de las variables ocultas (de todas las direcciones) de la última capa
        # por cada token en la secuencia de cada elemento del lote (num_steps, batch size, hid dim * n directions) .
        # rnn_outputs shape: (num_steps, batch size, hid dim) 
        # recuerde que state devolverá siempre el valor de las variables ocultas de todas las capas 
        # (y en cada dirección si hay mas de una) de cada elemento del lote. (n layers * n directions, batch_size, num_hiddens) 
        # state shape: (num_layers, batch_size, num_hiddens)   
        return output, state

Usemos un ejemplo concreto para ilustrar la implementación del encoder anterior. A continuación instanciamos un encoder GRU de dos capas cuyo número de unidades ocultas es 16. Dado un minilote de entradas de secuencia X (tamaño de lote: 4, número de pasos de tiempo: 9), los estados ocultos de la última capa en todos los pasos de tiempo (`outputs` devueltas por las capas recurrentes del encoder) son un tensor de forma (número de pasos de tiempo, tamaño del lote, número de unidades ocultas).

In [53]:
vocab_size, embed_size, num_hiddens, num_layers = 10, 8, 16, 2
batch_size, num_steps = 4, 9

encoder = RNNEncoder(vocab_size, embed_size, num_hiddens, num_layers)
X = torch.zeros((batch_size, num_steps))
outputs, state = encoder(X)
outputs.shape, (num_steps,batch_size,num_hiddens)

(torch.Size([4, 9, 16]), (9, 4, 16))

## Decoder 

Como acabamos de mencionar, la variable de contexto $\mathbf{c}$ de la salida del encoder codifica toda la secuencia de entrada $x_1, \ldots, x_T$. Dada la secuencia de salida $y_1, y_2, \ldots, y_{T'}$ del conjunto de datos de entrenamiento, para cada paso de tiempo $t'$ (el símbolo difiere del paso de tiempo $t$ de las secuencias de entrada del encoder), la probabilidad de que la salida del decoder $y_{t'}$ está condicionada a la subsecuencia de salida anterior $y_1, \ldots, y_{t'-1}$ y la variable de contexto $\mathbf{c}$, es decir, $P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \mathbf{c})$.

Para modelar esta probabilidad condicional en secuencias, podemos usar otro RNN como decoder. En cualquier paso de tiempo $t^\prime$ en la secuencia de salida, la RNN toma la salida $y_{t^\prime-1}$ del paso de tiempo anterior y la variable de contexto $\mathbf{c}$ como su entrada, luego los transforma junto con el estado oculto anterior $\mathbf{s}_{t^\prime-1}$ en el estado oculto $\mathbf{s}_{t^\prime}$ en el paso de tiempo actual. Como resultado, podemos usar una función $g$ para expresar la transformación de la capa oculta del decoder:

$$\mathbf{s}_{t^\prime} = g(y_{t^\prime-1}, \mathbf{c}, \mathbf{s}_{t^\prime-1}).$$

![Imgur](https://i.imgur.com/TwY4S1M.png)

Después de obtener el estado oculto del decoder, podemos usar una capa de salida y la operación softmax para calcular la distribución de probabilidad condicional $P(y_{t^\prime} \mid y_1, \ldots, y_{t^\prime-1}, \mathbf{c})$ para la salida en el paso de tiempo $t^\prime$.

Siguiendo la figura, al implementar el decoder de la siguiente manera, usamos directamente el estado oculto en el paso de tiempo final del encoder para inicializar el estado oculto del decoder. Esto requiere que el encoder RNN y el decoder RNN **tengan el mismo número de capas y unidades ocultas**. Para incorporar aún más la información de la secuencia de entrada codificada, la variable de contexto se concatena con la entrada del decoder en todos los pasos de tiempo. Para predecir la distribución de probabilidad del token de salida, se usa una capa densa para transformar el estado oculto en la capa final del decoder RNN.



In [54]:
class RNNDecoder(Decoder):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size+num_hiddens, num_hiddens,
                           num_layers, dropout=dropout)
        self.linear = nn.Linear(num_hiddens, vocab_size)
        self.hid_dim = num_hiddens
        self.n_layers = num_layers
        self.output_dim = vocab_size
        self.apply(init_RNN)

    def init_state(self, enc_output, *args):
        #se queda con el output del último paso de tiempo
        return enc_output[-1]

    def forward(self, X, context, hidden):
        # X shape: (batch_size)
        # context shape: (batch_size, num_hiddens)
        # hidden shape: (n layers, batch_size, num_hiddens)

        input = X.unsqueeze(0)
        context = context.unsqueeze(0)
        #input shape: (1,batch_size)
        #context shape: (1, batch_size, num_hiddens)

        embs = self.embedding(input)
        # embs shape: (1, batch_size, embed_size)

        # Concatena el token de entrada con el contexto
        embs_and_context = torch.cat((embs, context), -1)
        #embs_and_context shape: (1, batch size, embed_size + num_hiddens)
        
        #Genera las salidas de la RNN
        rnn_outputs, state = self.rnn(embs_and_context, hidden)
        # recuerde que output devolverá siempre el valor de las variables ocultas (de todas las direcciones) de la última capa
        # por cada token en la secuencia de cada elemento del lote (seq_len, batch size, hid dim * n directions) .
        # rnn_outputs shape: (1, batch size, hid dim) 
        # recuerde que state devolverá siempre el valor de las variables ocultas de todas las capas 
        # (y en cada dirección si hay mas de una) de cada elemento del lote. (n layers * n directions, batch_size, num_hiddens) 
        # state shape: (num_layers, batch_size, num_hiddens)       

        #Genera las probabilidades de cada token en el vocabulario
        outputs = self.linear(rnn_outputs.squeeze(0))
        #outputs = torch.cat((embs.squeeze(0), rnn_outputs.squeeze(0), context),dim = 1)

        # outputs shape: (batch_size, vocab_size)
        return outputs, state

## Teacher Forcing

Si bien la entrada del encoder son solo los tokens de la secuencia de origen, la entrada y la salida del decoder no son tan sencillas en el entrenamiento de un encoder-decoder. Una opción es alimentar al decoder con la salida del último paso de tiempo $y_{t-1}$ como entrada para el modelo en el paso de tiempo actual $X_t$. Esto causa que cada token predicho sea esencial para la predicción de los siguientes, aunque quizás puede resultar en una oración de salida completamente distinta si hay errores en los primeros tokens.

Otro enfoque es el Teacher Forcing (forzado del maestro), que es una estrategia para entrenar redes neuronales recurrentes que utiliza la etiqueta de la frase de destino como entrada, en lugar de la salida del modelo de un paso de tiempo anterior como entrada. El forzado del maestro funciona utilizando la salida real o esperada del conjunto de datos de entrenamiento en el paso de tiempo actual $y_t$ como entrada en el siguiente paso de tiempo $X_{t+1}$, en lugar de la salida generada por la red. Este enfoque tiene la ventaja de que errores al principio de la oración no condicionan la frase entera al entrenar, pero puede ocasionar que la red no sepa como sobreponerse a errores previos debido a que está acostumbrada a recibir siempre el token anterior correcto.

Quizás un enfoque intermedio sea alternar aleatoriamente entre ambas opciones como haremos en este notebook. Agregamos un hiperparámetro que gestiona la probabilidad de que se aplique teacher forcing.

In [55]:
import random
class Seq2Seq_TF(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "¡Las dimensiones ocultas del encoder y del decoder deben ser iguales!"
        assert encoder.n_layers == decoder.n_layers, \
            "¡El número de capas del encoder y del decoder deben ser iguales!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        enc_outputs, enc_hidden = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]

        context = self.decoder.init_state(enc_outputs)
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and context
            dec_output, dec_hidden = self.decoder(input, context, enc_hidden)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = dec_output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = dec_output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

## Función de Pérdida



En cada paso de tiempo, el decoder predice una distribución de probabilidad para los tokens de salida. Podemos aplicar softmax para obtener la distribución y calcular la pérdida de entropía cruzada para la optimización. 

![Imgur](https://i.imgur.com/Kd0z00x.png)

Recuerde de la clase de embeddings que tokens de relleno se agregan al final de las secuencias para que las secuencias de diferentes longitudes se puedan cargar de manera eficiente en minilotes de la misma forma. Sin embargo, la predicción de los tokens de relleno debe excluirse de los cálculos de pérdidas. En la clase de embeddings usamos máscaras para eliminar los rellenos, acá podemos usar el parámetro `ignore_index` de las pérdidas de torch para que ignore los tokens `<pad>`

In [56]:
# La siguiente celda no funcionará hasta que se cargue el vocabulario
# y se establezca el índice de padding
#loss = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

## Cargando los Datos



Vamos a entrenar un modelo que traduzca al español frases escritas en inglés. Para eso necesitamos un dataset que tenga frases escritas en ambos idiomas para usar como datasets de origen y destino.

Usaremos el dataset publicado por la organización [Tatoeba](https://tatoeba.org/es) que vamos a descargar en la siguiente celda.

In [57]:
import os.path
import re
from shutil import unpack_archive

data = None
!wget -O spa-eng.zip http://www.manythings.org/anki/spa-eng.zip
if not os.path.isfile("spa.txt"):
    unpack_archive('./spa-eng.zip', extract_dir='./', format='zip')
with open('./spa.txt', encoding='utf-8') as f:
    data = f.read()
    data = re.sub("\tCC-BY 2\.0.*","",data) # acá elimino información adicional
    data = re.sub(r"[\u202f]|[\xa0]"," ",data) # aca saco caracteres raros
    data = re.sub("([,\.:;!?])"," \\1",data) # aca  y abajo tokenizo puntuación
    data = re.sub("([¡¿])","\\1 ",data).lower()


--2022-08-27 16:19:41--  http://www.manythings.org/anki/spa-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5320329 (5.1M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   5.07M  7.12MB/s    in 0.7s    

2022-08-27 16:19:42 (7.12 MB/s) - ‘spa-eng.zip’ saved [5320329/5320329]



En la salida de la siguiente celda podemos ver la estructura de los datos: los tokens de cada frase están separados por un espacio y las frases están separadas por un tab.

In [58]:
print(data[:100])

go .	ve .
go .	vete .
go .	vaya .
go .	váyase .
hi .	hola .
run !	¡ corre !
run !	¡ corran !
run !	¡


Ahora tenemos que crear dos vocabularios para almacenar las frases en inglés y español respectivamente. En la siguiente celda separamos las frases usando el caracter `\t` que representa un tab. Luego agregamos los tokens especiales `<bos>` y `<eos>` que van a simbolizar el inicio y el final de oración respectivamente. El inicio de oración solo lo usamos en la frase de destino para ser usado como entrada en el paso de tiempo 1 del decoder (para saber que tiene que generar la primera palabra usando solo la información provista por el vector de contexto).  

In [59]:
import random

SRC_IDX, TGT_IDX = 0, 1

SEED = 12312

data2 = data.split('\n')
random.seed(SEED)
random.shuffle(data2)

data_list = []
for i, line in enumerate(data2):
    parts = line.split('\t')
    if len(parts) == 2:
        # Skip empty tokens
        new_src = [t for t in f'{parts[SRC_IDX]} <eos>'.split(' ') if t]
        new_tgt = [t for t in f'<bos> {parts[TGT_IDX]} <eos>'.split(' ') if t]
        length_src = len(new_src)
        data_list.append((new_src, length_src, new_tgt))

print(data_list[0][0], data_list[0][-1])
print(len(data_list))

['tom', 'seldom', 'puts', 'sugar', 'in', 'his', 'coffee', '.', '<eos>'] ['<bos>', 'tom', 'casi', 'nunca', 'le', 'pone', 'azúcar', 'al', 'café', '.', '<eos>']
138440


Ahora podemos crear los vocabularios de torchtext indicando todos los caracteres especiales (incluyendo '<unk>' para reemplazar las palabras que no aparecen en el vocabulario).

In [60]:
from collections import Counter
from torchtext.vocab import vocab

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

n = len(data_list)
split1, split2 = int(0.7*n), int(0.9*n)
train_list = data_list[:split1]
val_list = data_list[split1:split2]
test_list = data_list[split2:]

counter_src, counter_tgt = Counter(), Counter()
for i in range(len(train_list)):
  counter_src.update(train_list[i][0])
  counter_tgt.update(train_list[i][-1])

vocab_src = vocab(counter_src, min_freq = 2,
              specials=('<unk>', '<eos>', '<bos>', '<pad>'))
vocab_src.set_default_index(vocab_src['<unk>'])

vocab_tgt = vocab(counter_tgt, min_freq = 2,
              specials=('<unk>', '<eos>', '<bos>', '<pad>'))
vocab_tgt.set_default_index(vocab_tgt['<unk>'])


Las siguientes celdas llevan adelante el preprocesamiento necesario para generar los dataloaders al igual que lo hicimos en el ejercicio de la clase 1. En la siguiente celda `BucketSampler` es una clase que arma los minilotes con oraciones de longitud parecida de manera que el relleno necesario sea mínimo.


In [61]:
from torch.utils.data.sampler import Sampler

class BucketSampler(Sampler):

    def __init__(self, batch_size, train_list):
        self.length = len(train_list)
        self.train_list = train_list
        self.batch_size = batch_size
        indices = [(i, s[1]) for i, s in enumerate(self.train_list)]
        random.seed(SEED)
        random.shuffle(indices)
        pooled_indices = []
        # creamos minilotes de tamaños similares
        for i in range(0, len(indices), batch_size * 100):
            pooled_indices.extend(sorted(indices[i:i + batch_size * 100],
                                         key=lambda x: x[1], reverse=True))

        self.pooled_indices = pooled_indices

    def __iter__(self):
        for i in range(0, len(self.pooled_indices), self.batch_size):
            yield [idx for idx, _ in self.pooled_indices[i:i + self.batch_size]]

    def __len__(self):
        return (self.length + self.batch_size - 1) // self.batch_size

`collate_batch` es una función que le indica al DataLoader que datos devolver por cada ejemplo del Dataset. En este caso vamos a generar 3 lotes distintos en una tupla:
* text_src: son las frases en el idioma origen que le pasaremos como entrada al encoder
* text_tgt_in: son las frases en el idioma destino que le pasaremos como entrada al decoder (con `<bos>` como primer token y sin `<eos>`)
* text_tgt_out: son las frases en el idioma destino que usaremos para calcular la pérdida (con `<eos>` como finalizador de oración y sin `<bos>`)

In [62]:
import torch

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

SRC_PAD_IDX = vocab_src['<pad>']
TGT_PAD_IDX = vocab_tgt['<pad>']

def collate_batch(batch):
    text_src, length_list, text_tgt_in, text_tgt_out = [], [], [], []
    for (src, length, tgt) in batch:
        # convertimos el texto en tokens
        processed_src = torch.tensor([vocab_src[token] for token in src])
        processed_tgt = torch.tensor([vocab_tgt[token] for token in tgt])
        text_src.append(processed_src)
        text_tgt_in.append(processed_tgt[:-1])
        text_tgt_out.append(processed_tgt[1:])
        # guardamos la longitud de cada token
        length_list.append(length)
    # armamos la tupla que conformara un ejemplo de minilote.
    result = (pad_sequence(text_src, padding_value=SRC_PAD_IDX),
              pad_sequence(text_tgt_in, padding_value=TGT_PAD_IDX),
              pad_sequence(text_tgt_out, padding_value=TGT_PAD_IDX),)
    return result

Ahora podemos crear los DataLoaders e imprimir su contenido.

In [63]:
batch_size = 64  # A batch size of 64

train_bucket = BucketSampler(batch_size, train_list)
train_iter = DataLoader(train_list,
                          batch_sampler=train_bucket,
                          collate_fn=collate_batch)

val_bucket = BucketSampler(batch_size, val_list)
val_iter = DataLoader(val_list,
                          batch_sampler=val_bucket,
                          collate_fn=collate_batch)

test_bucket = BucketSampler(batch_size, test_list)
test_iter = DataLoader(test_list,
                          batch_sampler=test_bucket,
                          collate_fn=collate_batch)

In [64]:
src, input, out = next(iter(test_iter))

a = [vocab_src.get_itos()[token] for example in src.T[:1] for token in example]
b = [vocab_tgt.get_itos()[token] for example in out.T[:1] for token in example]
c = [vocab_tgt.get_itos()[token] for example in input.T[:1] for token in example]

print(a)
print(b)
print(c)

['in', '1969', ',', 'roger', 'miller', '<unk>', 'a', 'song', 'called', '"you', "don't", 'want', 'my', 'love', '."', 'today', ',', 'this', 'song', 'is', 'better', 'known', 'as', '<unk>', 'the', 'summer', 'time', '."', "it's", 'the', 'first', 'song', 'he', 'wrote', 'and', 'sang', 'that', 'became', 'popular', '.', '<eos>']
['en', '1969', ',', 'roger', 'miller', 'grabó', 'una', 'canción', 'llamada', '<unk>', 'no', 'quieres', 'mi', '<unk>', '.', 'hoy', ',', 'esta', 'canción', 'es', 'más', 'conocida', 'como', '"en', 'el', '<unk>', '.', 'es', 'la', 'primera', 'canción', 'que', 'escribió', 'y', 'cantó', 'que', 'se', 'convirtió', 'popular', '.', '<eos>']
['<bos>', 'en', '1969', ',', 'roger', 'miller', 'grabó', 'una', 'canción', 'llamada', '<unk>', 'no', 'quieres', 'mi', '<unk>', '.', 'hoy', ',', 'esta', 'canción', 'es', 'más', 'conocida', 'como', '"en', 'el', '<unk>', '.', 'es', 'la', 'primera', 'canción', 'que', 'escribió', 'y', 'cantó', 'que', 'se', 'convirtió', 'popular', '.']


## Bucle de Entrenamiento

Ahora que ya tenemos los datos cargados, entrenaremos nuestro modelo.

In [65]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src, tgt_input, tgt_out = batch
        src, tgt_input, tgt_out = src.to(device), tgt_input.to(device), tgt_out.to(device)        
        #src: son las frases en el idioma origen que le pasaremos como entrada al encoder
        #src.shape : [src len, batch size]
        #tgt_input: son las frases en el idioma destino que le pasaremos como entrada al decoder (con `<bos>` como primer token y sin `<eos>`)
        #tgt_out: son las frases en el idioma destino que usaremos para calcular la pérdida (con `<eos>` como finalizador de oración y sin `<bos>`)
        #tgt.shape : [trg len, batch size]
        
        optimizer.zero_grad()
        output = model(src, tgt_input)
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        #como la función de pérdida solo funciona en entradas 2d con objetivos 1d,
        # necesitamos aplanar cada una de ellas con .view
        output = output[1:].view(-1, output_dim)
        trg = tgt_out[1:].view(-1)
        #trg = [trg len * batch size]
        #output = [trg len * batch size, output dim]
        
        #calculamos los gradientes
        loss = criterion(output, trg)
        loss.backward()

        #recortamos los gradientes para evitar que exploten
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

El loop de evaluación es similar al de entranmiento (sin las actualizaciones de los parámetros), sin embargo, debemos asegurarnos de desactivar el forzamiento del maestro para la evaluación. Esto hará que el modelo solo use sus propias predicciones para hacer más predicciones dentro de una oración, lo que refleja cómo se usaría en producción.

Otras diferencias incluyen:
* configurar el modelo en modo de evaluación con `model.eval()`. Esto desactivará el dropout (y la normalización de lotes, si se usa).

* Usar el bloque `with torch.no_grad()` para asegurarnos de que no se calculen gradientes dentro del bloque. Esto reduce el consumo de memoria y acelera las cosas.

In [66]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0 
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src, tgt_input, tgt_out = batch
            src, tgt_input, tgt_out = src.to(device), tgt_input.to(device), tgt_out.to(device)    
            output = model(src, tgt_input, 0) #turn off teacher forcing
            #output = [trg len, batch size, output dim]
            output_dim = output.shape[-1]

            output = output.view(-1, output_dim)
            trg = tgt_out.view(-1)
            #trg = [trg len * batch size]
            #output = [trg len * batch size, output dim]

            loss = criterion(output, trg)            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

A continuación, crearemos una función que usaremos para decirnos cuánto tarda una época.

In [67]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Por último, definimos el modelo, la pérdida y el optimizador.

In [68]:
import torch.optim as optim

INPUT_DIM = len(vocab_src.get_itos())
OUTPUT_DIM = len(vocab_tgt.get_itos())
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = RNNEncoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = RNNDecoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq_TF(enc, dec, device).to(device)

model = Seq2Seq_TF(enc, dec, device).to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = TGT_PAD_IDX)

Luego, entrenamos nuestro modelo, guardando los parámetros que nos den la mejor pérdida de validación.

In [69]:
import math
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_iter, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 48s
	Train Loss: 4.695 | Train PPL: 109.435
	 Val. Loss: 4.624 |  Val. PPL: 101.886
Epoch: 02 | Time: 1m 47s
	Train Loss: 3.298 | Train PPL:  27.047
	 Val. Loss: 4.222 |  Val. PPL:  68.165
Epoch: 03 | Time: 1m 47s
	Train Loss: 2.801 | Train PPL:  16.463
	 Val. Loss: 4.066 |  Val. PPL:  58.294
Epoch: 04 | Time: 1m 47s
	Train Loss: 2.548 | Train PPL:  12.776
	 Val. Loss: 4.056 |  Val. PPL:  57.760
Epoch: 05 | Time: 1m 47s
	Train Loss: 2.415 | Train PPL:  11.186
	 Val. Loss: 4.053 |  Val. PPL:  57.595
Epoch: 06 | Time: 1m 47s
	Train Loss: 2.335 | Train PPL:  10.331
	 Val. Loss: 4.092 |  Val. PPL:  59.883
Epoch: 07 | Time: 1m 47s
	Train Loss: 2.282 | Train PPL:   9.801
	 Val. Loss: 4.085 |  Val. PPL:  59.419
Epoch: 08 | Time: 1m 47s
	Train Loss: 2.252 | Train PPL:   9.508
	 Val. Loss: 4.088 |  Val. PPL:  59.610
Epoch: 09 | Time: 1m 47s
	Train Loss: 2.233 | Train PPL:   9.328
	 Val. Loss: 4.121 |  Val. PPL:  61.636
Epoch: 10 | Time: 1m 47s
	Train Loss: 2.212 | Train PPL

In [70]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.067 | Test PPL:  58.395 |
